In [ ]:
!pip install geopandas
!python -m pip install statsmodels
!pip install folium

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

In [ ]:
column_headers = [
    "Transaction_unique_identifier",
    "Price",
    "Date_of_Transfer",
    "Postcode",
    "Property_Type",
    "Old/New",
    "Duration",
    "PAON",
    "SAON",
    "Street",
    "Locality",
    "Town/City",
    "District",
    "County",
    "PPD_Category_Type",
    "Record_Status_monthly_file_only"
]

In [ ]:
df_main = pd.read_csv('/content/drive/MyDrive/Chimnie/pp-complete_2.csv',names = column_headers)

In [ ]:
df = df_main.copy()

In [ ]:
# Convert columns to appropriate datatypes
df['Transaction_unique_identifier'] = df['Transaction_unique_identifier'].astype('string')
df['Price'] = df['Price'].astype('float')
df['Date_of_Transfer'] = pd.to_datetime(df['Date_of_Transfer'])
df['Postcode'] = df['Postcode'].astype('string')
df['Property_Type'] = df['Property_Type'].astype('category')
df['Old/New'] = df['Old/New'].astype('category')
df['Duration'] = df['Duration'].astype('category')
df['PAON'] = df['PAON'].astype('string')
df['SAON'] = df['SAON'].astype('string')
df['Street'] = df['Street'].astype('string')
df['Locality'] = df['Locality'].astype('string')
df['Town/City'] = df['Town/City'].astype('string')
df['District'] = df['District'].astype('string')
df['County'] = df['County'].astype('string')
df['PPD_Category_Type'] = df['PPD_Category_Type'].astype('category')
df['Record_Status_monthly_file_only'] = df['Record_Status_monthly_file_only'].astype('category')

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
nspl_df_main = pd.read_csv('/content/drive/MyDrive/Chimnie/NSPL_2021_MAY_2024/Data/NSPL21_MAY_2024_UK.csv')

In [ ]:
nspl_df = nspl_df_main.copy()

In [ ]:
nspl_df['pcd'] = nspl_df['pcd'].astype('string')
nspl_df['pcd2'] = nspl_df['pcd2'].astype('string')
nspl_df['pcds'] = nspl_df['pcds'].astype('string')
nspl_df['laua'] = nspl_df['laua'].astype('string')
nspl_df['rgn'] = nspl_df['rgn'].astype('string')

In [ ]:
nspl_df.isna().sum()

In [ ]:
nspl_df.info()

In [ ]:
# Remove all spaces in the 'pcd', 'pcd2', and 'pcds' columns
nspl_df['pcd'] = nspl_df['pcd'].str.replace(r'\s+', '', regex=True)
nspl_df['pcd2'] = nspl_df['pcd2'].str.replace(r'\s+', '', regex=True)
nspl_df['pcds'] = nspl_df['pcds'].str.replace(r'\s+', '', regex=True)

In [ ]:
len(nspl_df[(nspl_df['pcd'] == nspl_df['pcd2']) & (nspl_df['pcd2'] == nspl_df['pcds'])])

# Exercise 2


In [ ]:
london_boroughs = nspl_df[nspl_df['laua'].str.startswith('E09')]

In [ ]:
len(london_boroughs)

In [ ]:
df['Postcode'] = df['Postcode'].str.replace(r'\s+', '', regex=True)

In [ ]:
df_2023 = df[df['Date_of_Transfer'].dt.year == 2023]

# Merge with NSPL dataset to get borough information
filtered_df_2023 = df_2023[df_2023['Postcode'].isin(london_boroughs['pcd'])]


In [ ]:
borough_sales = filtered_df_2023.groupby('District').agg(
    Sales_Count=('Price', 'size'),
    Average_Price=('Price', 'mean')
).reset_index()

In [ ]:
borough_sales['District'] = borough_sales['District'].replace('CITY OF WESTMINSTER', 'WESTMINSTER')

borough_sales

In [ ]:
gdf = gpd.read_file('/content/drive/MyDrive/Chimnie/london_boroughs.geojson')

In [ ]:
gdf['name'] = gdf['name'].str.upper()
gdf

In [ ]:
merged_gdf = gdf.set_index('name').join(borough_sales.set_index('District'))

# Plotting
fig, ax = plt.subplots(1, 1, figsize=(20, 15))
merged_gdf.plot(column='Average_Price', cmap='Accent', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)

# Add titles and labels
ax.set_title('Average Price in London Borough - 2023', fontdict={'fontsize': '10', 'fontweight': '5'})
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)


offsets = {
    'KENSINGTON AND CHELSEA': (0, -3),
    'HAMMERSMITH AND FULHAM': (0, -12)
}
for idx, row in merged_gdf.iterrows():
    offset = offsets.get(row.name, (0, 0))
    plt.annotate(
        text=f"{row.name}\n{row.Average_Price:.2f}",
        xy=(row.geometry.centroid.x, row.geometry.centroid.y),
        xytext=offset,
        textcoords='offset points',
        ha='center',
        va='center',
        fontsize=7,
        color='black',
        bbox=dict(facecolor='white', alpha=0.5, edgecolor='none', pad=1)  # Optional for better readability
    )

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 15))
merged_gdf.plot(column='Sales_Count', cmap='spring', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)

ax.set_title('Sales Count in London Borough - 2023', fontdict={'fontsize': '10', 'fontweight': '5'})
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)


offsets = {
    'HAMMERSMITH AND FULHAM': (0, -12)
}
for idx, row in merged_gdf.iterrows():
    offset = offsets.get(row.name, (0, 0))
    plt.annotate(
        text=f"{row.name}\n{row.Sales_Count}",
        xy=(row.geometry.centroid.x, row.geometry.centroid.y),
        xytext=offset,
        textcoords='offset points',
        ha='center',
        va='center',
        fontsize=7,
        color='black',
        bbox=dict(facecolor='white', alpha=0.5, edgecolor='none', pad=1)  # Optional for better readability
    )


plt.show()

# Exercise 3


In [ ]:
#filter year from 2020
df_from_2020 = df[df['Date_of_Transfer'].dt.year >= 2020]

#filter Property_Type as 'F' (Flat) and it should be new
df_from_2020 = df_from_2020[(df_from_2020['Property_Type'] == 'F') & (df_from_2020['Old/New'] == 'Y')]


In [ ]:
df_from_2020.describe()

In [ ]:
region_df = nspl_df.copy()
region_df = region_df.dropna(subset=['rgn'])

def determine_region(rgn):
    if rgn.startswith('E12'):
        return 'England'
    elif rgn.startswith('W'):
        return 'Wales'
    elif rgn.startswith('S'):
        return 'Scotland'
    elif rgn.startswith('N'):
        return 'Northern Ireland'
    elif rgn.startswith('L'):
        return 'Channel Islands'
    elif rgn.startswith('M'):
        return 'Isle of Man'
    else:
        return 'Unknown'

# Apply the function to the 'rgn' column to create the 'Region' column
region_df['Region'] = region_df['rgn'].apply(determine_region)

In [ ]:
region_df_from_2020 = df_from_2020.merge(region_df, left_on='Postcode', right_on='pcd', how='left')

In [ ]:
region_df_from_2020['Region'].value_counts()

Postcode lookup has 6 Regions:
England

1.   England
2.   Wales

1.   Scotland
2.   Northern Ireland

1.   Channel Islands
2.   Isle of Man

But the Price Paid Data contains only the property sales in **England** and **Wales**

In [ ]:
region_sales = region_df_from_2020.groupby('Region').agg(
    New_Build_Flats_Sold=('Transaction_unique_identifier', 'count')
).reset_index()

print(region_sales)

In [ ]:
region_df_from_2020['Region'].value_counts()

# Exercise 4

In [ ]:
df_2020_onwards = df[df['Date_of_Transfer'] >= '2020-01-01']

df_2020_onwards['Week'] = df_2020_onwards['Date_of_Transfer'].dt.to_period('W').astype(str)


In [ ]:
# Count the number of sales per week
weekly_sales = df_2020_onwards.groupby('Week').size().reset_index(name='Sales_Count')

In [ ]:
weekly_sales['Week'].nunique()

In [ ]:
# Plot a line chart of sales per week
plt.figure(figsize=(15, 8))
plt.plot(weekly_sales['Week'], weekly_sales['Sales_Count'], marker='o')
plt.title('Number of Sales Per Week Since the Start of 2020')
plt.xlabel('Week')
plt.ylabel('Number of Sales')

# Set x-ticks at intervals
num_weeks = len(weekly_sales['Week'])
plt.xticks(weekly_sales['Week'][::5], rotation=90)  # Adjust interval as needed

plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
avg_price_per_week = df_2020_onwards.groupby('Week')['Price'].mean().reset_index(name='Average_Price')

In [ ]:
# Plot a line chart of sales per week
plt.figure(figsize=(15, 8))
plt.plot(avg_price_per_week['Week'], avg_price_per_week['Average_Price'], marker='o')
plt.title('Avg price Per Week Since the Start of 2020')
plt.xlabel('Week')
plt.ylabel('Avg Price')

# Set x-ticks at intervals
num_weeks = len(weekly_sales['Week'])
plt.xticks(weekly_sales['Week'][::5], rotation=90)  # Adjust interval as needed

plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Create a figure and a set of subplots
fig, ax1 = plt.subplots(figsize=(15, 8))

# Plot the average price per week on the left y-axis
ax1.plot(avg_price_per_week['Week'], avg_price_per_week['Average_Price'], marker='o', color='blue', label='Average Price')
ax1.set_xlabel('Week')
ax1.set_ylabel('Average Price', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Set x-ticks at intervals
ax1.set_xticks(avg_price_per_week['Week'][::5])
ax1.set_xticklabels(avg_price_per_week['Week'][::5], rotation=90)  # Adjust interval as needed

# Create a second y-axis sharing the same x-axis
ax2 = ax1.twinx()
ax2.plot(weekly_sales['Week'], weekly_sales['Sales_Count'], marker='o', color='green', label='Number of Sales')
ax2.set_ylabel('Number of Sales', color='green')
ax2.tick_params(axis='y', labelcolor='green')

# Set title and layout
plt.title('Average Price and Number of Sales Per Week Since the Start of 2020')
fig.tight_layout()

# Show plot
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Calculate the moving average
weekly_sales['Sales_MA'] = weekly_sales['Sales_Count'].rolling(window=4).mean()

# Plot the data with moving average
plt.figure(figsize=(14, 7))
plt.plot(weekly_sales['Week'], weekly_sales['Sales_Count'], marker='o', label='Weekly Sales')
plt.plot(weekly_sales['Week'], weekly_sales['Sales_MA'], color='red', label='4-Week Moving Average')
plt.title('Number of Sales Per Week Since the Start of 2020 with Moving Average')
plt.xlabel('Week')
plt.ylabel('Number of Sales')
plt.xticks(weekly_sales['Week'][::5], rotation=90)  # Adjust interval as needed
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Convert 'Week' to datetime format for decomposition
weekly_sales['Week_Start'] = pd.to_datetime(weekly_sales['Week'].apply(lambda x: x.split('/')[0]))

# Perform seasonal decomposition
decomposition = seasonal_decompose(weekly_sales.set_index('Week_Start')['Sales_Count'], model='additive', period=52)

# Plot the decomposition
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(14, 12))
decomposition.observed.plot(ax=ax1, title='Observed')
decomposition.trend.plot(ax=ax2, title='Trend')
decomposition.seasonal.plot(ax=ax3, title='Seasonal')
decomposition.resid.plot(ax=ax4, title='Residual')
plt.tight_layout()
plt.show()


In [ ]:
backup_weekly_sales= weekly_sales.copy()

# Exercise 5


In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df['Price'], bins=20, edgecolor='black')
plt.title('Distribution of Sale Prices')
plt.xlabel('Sale Price')
plt.ylabel('Frequency')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np

# Calculate statistics
mean_price = np.mean(df['Price'])
median_price = np.median(df['Price'])
std_dev_price = np.std(df['Price'])
range_price = np.ptp(df['Price'])
skewness = df['Price'].skew()
kurtosis = df['Price'].kurt()

# Print the statistics
print(f'Mean Sale Price: {mean_price}')
print(f'Median Sale Price: {median_price}')
print(f'Standard Deviation of Sale Prices: {std_dev_price}')
print(f'Range of Sale Prices: {range_price}')
print(f'Skewness of Sale Prices: {skewness}')
print(f'Kurtosis of Sale Prices: {kurtosis}')


In [ ]:
plt.figure(figsize=(10, 6))
plt.boxplot(df['Price'])
plt.title('Box Plot of Price')
plt.xlabel('Price')
plt.show()

In [ ]:
price_99_percentile = df['Price'].quantile(0.99)

# Filter the DataFrame to include only data up to the 99th percentile
filtered_df = df[df['Price'] <= price_99_percentile]

# Plot the box plot
plt.figure(figsize=(10, 6))
plt.boxplot(filtered_df['Price'])
plt.title('Box Plot of Price up to the 99th Percentile')
plt.xlabel('Price')
plt.show()

In [ ]:
# Apply log transformation to Sale_Price
df['Log_Sale_Price'] = np.log(df['Price'])

plt.figure(figsize=(12, 6))
sns.histplot(df['Log_Sale_Price'], kde=True, bins=30)
plt.title('Distribution of Log-Transformed Sale Prices')
plt.xlabel('Log Sale Price')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Define a threshold to identify outliers (e.g., values above the 99th percentile)
threshold = df['Price'].quantile(0.99)
df_filtered = df[df['Price'] < threshold]

# Plot the histogram of the filtered sale prices
plt.figure(figsize=(10, 6))
plt.hist(df_filtered['Price'], bins=20, edgecolor='black')
plt.title('Distribution of Sale Prices (Filtered)')
plt.xlabel('Sale Price')
plt.ylabel('Frequency')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np

# Log-Transformed Data Analysis
mean_log_price = np.mean(df['Log_Sale_Price'])
median_log_price = np.median(df['Log_Sale_Price'])
std_dev_log_price = np.std(df['Log_Sale_Price'])
skewness_log = df['Log_Sale_Price'].skew()
kurtosis_log = df['Log_Sale_Price'].kurt()

print(f'Mean Log Sale Price: {mean_log_price}')
print(f'Median Log Sale Price: {median_log_price}')
print(f'Standard Deviation of Log Sale Prices: {std_dev_log_price}')
print(f'Skewness of Log Sale Prices: {skewness_log}')
print(f'Kurtosis of Log Sale Prices: {kurtosis_log}')

# Filtered Data Analysis
mean_filtered_price = np.mean(df_filtered['Price'])
median_filtered_price = np.median(df_filtered['Price'])
std_dev_filtered_price = np.std(df_filtered['Price'])
skewness_filtered = df_filtered['Price'].skew()
kurtosis_filtered = df_filtered['Price'].kurt()

print(f'Mean Filtered Sale Price: {mean_filtered_price}')
print(f'Median Filtered Sale Price: {median_filtered_price}')
print(f'Standard Deviation of Filtered Sale Prices: {std_dev_filtered_price}')
print(f'Skewness of Filtered Sale Prices: {skewness_filtered}')
print(f'Kurtosis of Filtered Sale Prices: {kurtosis_filtered}')


# Exercise 6 & 7


In [ ]:
gdf1 = gpd.read_file('/content/drive/MyDrive/Chimnie/OSGB_Grids-master/GeoJSON/OSGB_Grid_10km.geojson')

In [ ]:
gdf1.info()

In [ ]:
print(gdf1.crs)

In [ ]:
gdf1.head()

In [ ]:
first_geometry = gdf1.loc[0, 'geometry']
print(f"Geometry Type: {first_geometry.geom_type}")
print(f"Geometry Coordinates: {first_geometry}")

# Loop through the first few geometries and print their types and coordinates
print(gdf1.loc[0, 'geometry'].__geo_interface__)


In [ ]:
df['Postcode'] = df['Postcode'].str.replace(r'\s+', '', regex=True)

In [ ]:
df_new = df.merge(nspl_df[['pcd', 'lat', 'long', 'oseast1m','osnrth1m']],left_on='Postcode', right_on='pcd', how='left')

In [ ]:
df_new.head()

In [ ]:
print(df_new[['lat','long']].isna().sum())

df_new = df_new.dropna(subset=['lat'])
df_new.isna().sum()

In [ ]:
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(df_new['long'], df_new['lat'])]
new_gdf = gpd.GeoDataFrame(df_new, geometry=geometry)

In [ ]:
new_gdf.set_crs(epsg=4326, inplace=True)
gdf1.set_crs(epsg=4326, inplace=True)

In [ ]:
new_gdf = gpd.sjoin(new_gdf, gdf1[['TILE_NAME', 'geometry']], how='left', predicate='within')

In [ ]:
new_gdf.head()

In [ ]:
new_gdf['TILE_NAME'].value_counts()

In [ ]:
sales_gdf = new_gdf.groupby('TILE_NAME').agg(
    total_sales=('Transaction_unique_identifier', 'count'),
    Average_Price=('Price', 'mean')
).reset_index()


print(sales_gdf)

In [ ]:
merged_gdf = gdf1.merge(sales_gdf, left_on = 'TILE_NAME', right_on='TILE_NAME', how='left')

In [ ]:
#merged_gdf = merged_gdf.to_crs('EPSG:27700')
merged_gdf.crs

In [ ]:
merged_gdf.head()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(30, 30))
merged_gdf.plot(column='total_sales', cmap='OrRd', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)
#merged_gdf.plot(column='REGION', ax=ax, legend=True, cmap='Set3', edgecolor='k', linewidth=0.8)
ax.set_title('total_sales', fontdict={'fontsize': '20', 'fontweight': '5'})
ax.set_xlabel('Longitude', fontsize=15)
ax.set_ylabel('Latitude', fontsize=15)


for idx, row in merged_gdf.iterrows():
    offset = offsets.get(row.name, (0, 0))
    plt.annotate(
        text=f"{row.total_sales}",
        xy=(row.geometry.centroid.x, row.geometry.centroid.y),
        xytext=offset,
        textcoords='offset points',
        ha='center',
        va='center',
        fontsize=5,
        color='black',
        bbox=dict(facecolor='white', alpha=0.5, edgecolor='none', pad=1)
    )



In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(30, 30))
merged_gdf.plot(column='Average_Price', cmap='OrRd', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)
#merged_gdf.plot(column='REGION', ax=ax, legend=True, cmap='Set3', edgecolor='k', linewidth=0.8)
ax.set_title('Average Price', fontdict={'fontsize': '20', 'fontweight': '5'})
ax.set_xlabel('Longitude', fontsize=15)
ax.set_ylabel('Latitude', fontsize=15)


for idx, row in merged_gdf.iterrows():
    offset = offsets.get(row.name, (0, 0))
    plt.annotate(
        text=f"{row.Average_Price}",
        xy=(row.geometry.centroid.x, row.geometry.centroid.y),
        xytext=offset,
        textcoords='offset points',
        ha='center',
        va='center',
        fontsize=5,
        color='black',
        bbox=dict(facecolor='white', alpha=0.5, edgecolor='none', pad=1)
    )



In [ ]:
def determine_region(row):
    if row['ENGLAND'] == 't':  # Assuming '1' indicates presence
        return 'England'
    elif row['SCOTLAND'] == 't':  # Assuming '1' indicates presence
        return 'Scotland'
    elif row['WALES'] == 't':  # Assuming '1' indicates presence
        return 'Wales'
    else:
        return 'Other'

merged_gdf['REGION'] = merged_gdf.apply(determine_region, axis=1)


bounds = [[49.5, -10.5], [61, 2]]  # Approximate bounds for the UK



# Create a folium map centered around the bounds of the UK
m = folium.Map(location=[55, -3], zoom_start=6, max_bounds=True, tiles='CartoDB positron')
m.fit_bounds(bounds)

# Add polygons to the map
folium.GeoJson(
    merged_gdf.to_json(),
    name='Polygons',
    style_function=lambda x: {

        'weight': 1,
        'fillOpacity': 0.1,
    },
    tooltip=folium.GeoJsonTooltip(fields=['TILE_NAME', 'total_sales','Average_Price'])
).add_to(m)

# Prepare data for heat map
heat_data = [[point.xy[1][0], point.xy[0][0], row['total_sales']]
             for index, row in merged_gdf.iterrows()
             for point in [row['geometry'].centroid]
             if not pd.isnull(row['total_sales'])]

# Add heat map
HeatMap(heat_data).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Save the map to an HTML file and display it
m.save('map.html')
m

# Exercise 9

In [ ]:
def determine_region(row):
    if row['ENGLAND'] == 't':  # Assuming '1' indicates presence
        return 'England'
    elif row['SCOTLAND'] == 't':  # Assuming '1' indicates presence
        return 'Scotland'
    elif row['WALES'] == 't':  # Assuming '1' indicates presence
        return 'Wales'
    else:
        return 'Other'

gdf1['REGION'] = gdf1.apply(determine_region, axis=1)

# Plotting the GeoDataFrame with different colors for each region
fig, ax = plt.subplots(figsize=(10, 10))
gdf1.plot(column='REGION', ax=ax, legend=True, cmap='Set3', edgecolor='k')

# Setting title and labels
ax.set_title('10km Map of England, Scotland, and Wales')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Show plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot the first five geometries
gdf1.head(2).plot()
plt.title('First Five Geometries')
plt.show()


In [ ]:
gdf1.loc[1,'geometry']

gdf1.loc[[1]].plot()
plt.title('Polygon for Record 0')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
weekly_sales= backup_weekly_sales.copy()

In [ ]:
from statsmodels.tsa.stattools import adfuller
weekly_sales= backup_weekly_sales.copy()

weekly_sales = weekly_sales.drop('Sales_MA', axis=1)
# Set the index to Week_Start for time series analysis
weekly_sales.set_index('Week_Start', inplace=True)

# Perform the Augmented Dickey-Fuller test
result = adfuller(weekly_sales['Sales_Count'].dropna())
print('ADF Statistic:', result[0])
print('p-value:', result[1])


In [ ]:
import matplotlib.pyplot as plt

# Plot with EPSG:4326
fig, ax = plt.subplots(1, 2, figsize=(15, 8))

gdf_4326 = gdf1.to_crs(epsg=4326)
gdf_4326.plot(ax=ax[0], edgecolor='k')
ax[0].set_title('WGS 84 (EPSG:4326)')

# Plot with EPSG:27700
gdf_27700 = gdf1.to_crs(epsg=27700)
gdf_27700.plot(ax=ax[1], edgecolor='k')
ax[1].set_title('British National Grid (EPSG:27700)')

plt.show()


In [ ]:
weekly_sales['Sales_Count_Diff'] = weekly_sales['Sales_Count'].diff().dropna()

In [ ]:
import itertools
import warnings
warnings.filterwarnings("ignore")

p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 52) for x in pdq]

best_aic = float("inf")
best_pdq = None
best_seasonal_pdq = None
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = SARIMAX(weekly_sales['Sales_Count'], order=param, seasonal_order=param_seasonal, enforce_stationarity=False, enforce_invertibility=False)
            results = mod.fit()
            if results.aic < best_aic:
                best_aic = results.aic
                best_pdq = param
                best_seasonal_pdq = param_seasonal
        except:
            continue

print(f'Best ARIMA Parameters: {best_pdq} with seasonal {best_seasonal_pdq}')

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

weekly_sales= backup_weekly_sales.copy()

weekly_sales = weekly_sales.drop('Sales_MA', axis=1)
# Set the index to Week_Start for time series analysis
weekly_sales.set_index('Week_Start', inplace=True)


# Fit the ARIMA model (order parameters can be tuned)
arima_model = ARIMA(weekly_sales['Sales_Count'], order=(5, 1, 2))
arima_result = arima_model.fit()

# Forecast future sales

# Forecast future sales
forecast = arima_result.get_forecast(steps=52)  # Forecast for one year
forecast_index = pd.date_range(start=weekly_sales.index[-1], periods=52, freq='W-MON')
forecast_series = pd.Series(forecast.predicted_mean, index=forecast_index)

# Plot the original data and forecast
plt.figure(figsize=(14, 7))
plt.plot(weekly_sales['Sales_Count'], label='Observed')
plt.plot(forecast_series, color='red', label='Forecast')
plt.title('Sales Forecast using ARIMA')
plt.xlabel('Date')
plt.ylabel('Number of Sales')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA

# Load the data (assuming weekly_sales DataFrame with 'Week_Start' as datetime and 'Sales_Count')
weekly_sales = backup_weekly_sales.copy()
weekly_sales = weekly_sales.drop('Sales_MA', axis=1)
weekly_sales = weekly_sales.join(avg_price_per_week['Average_Price'], how='left')

weekly_sales.set_index('Week_Start', inplace=True)
weekly_sales

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

endog = weekly_sales['Sales_Count']
exog = weekly_sales[['Average_Price']]

# Fit the SARIMA model on the training set
sarima_model = SARIMAX(endog, exog=exog, order=(2, 1, 2), seasonal_order=(1, 1, 1, 52))
sarima_result = sarima_model.fit(disp=False)


# Forecast future sales with exogenous variables
forecast_steps = 52  # Forecast for one year
future_exog = exog.iloc[-1:].append(pd.DataFrame({'Average_Price': exog['Average_Price'].iloc[-1]}, index=pd.date_range(start=exog.index[-1] + pd.Timedelta(weeks=1), periods=forecast_steps, freq='W-MON')))

forecast = sarima_result.get_forecast(steps=forecast_steps, exog=future_exog)
forecast_index = pd.date_range(start=weekly_sales.index[-1] + pd.Timedelta(weeks=1), periods=forecast_steps, freq='W-SUN')
forecast_series = pd.Series(forecast.predicted_mean, index=forecast_index)
forecast_ci = forecast.conf_int()

# Plot the original data and forecast
plt.figure(figsize=(14, 7))
plt.plot(weekly_sales['Sales_Count'], label='Observed')
plt.plot(forecast_series, color='red', label='Forecast')
plt.fill_between(forecast_index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], color='pink', alpha=0.3)
plt.title('Sales Forecast using SARIMA with Exogenous Variable (Average Price)')
plt.xlabel('Date')
plt.ylabel('Number of Sales')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Print the forecasted values for verification
print(forecast_series)


In [ ]:
# Diagnose the model
print(sarima_result.summary())
residuals = sarima_result.resid
plt.figure(figsize=(10, 6))
plt.plot(residuals)
plt.title('Residuals of ARIMA Model')
plt.show()

# Plot ACF of residuals to check if they resemble white noise
plot_acf(residuals, lags=50)
plt.show()

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Fit the SARIMA model (with seasonal components if necessary)
sarima_model = SARIMAX(weekly_sales['Sales_Count'], order=(2, 1, 2), seasonal_order=(1, 1, 1, 52))
sarima_result = sarima_model.fit(disp=False)

# Forecast future sales
forecast_steps = 52  # Forecast for one year
forecast = sarima_result.get_forecast(steps=forecast_steps)
forecast_index = pd.date_range(start=weekly_sales.index[-1] + pd.Timedelta(weeks=1), periods=forecast_steps, freq='W-MON')
forecast_series = pd.Series(forecast.predicted_mean, index=forecast_index)
forecast_ci = forecast.conf_int()

# Plot the original data and forecast
plt.figure(figsize=(14, 7))
plt.plot(weekly_sales['Sales_Count'], label='Observed')
plt.plot(forecast_series, color='red', label='Forecast')
plt.fill_between(forecast_index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], color='pink', alpha=0.3)
plt.title('Sales Forecast using SARIMA')
plt.xlabel('Date')
plt.ylabel('Number of Sales')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Diagnose the model
print(sarima_result.summary())
residuals = sarima_result.resid
plt.figure(figsize=(10, 6))
plt.plot(residuals)
plt.title('Residuals of ARIMA Model')
plt.show()

# Plot ACF of residuals to check if they resemble white noise
plot_acf(residuals, lags=50)
plt.show()

In [ ]:
train

In [ ]:
# Split the data into train and test sets
train_size = int(len(weekly_sales) * 0.8)
train, test = weekly_sales.iloc[:train_size], weekly_sales.iloc[train_size:]


# Fit the SARIMA model on the training set
sarima_model = SARIMAX(train['Sales_Count'], order=(3, 1, 2), seasonal_order=(1, 1, 1, 52))
sarima_result = sarima_model.fit(disp=False)

# Forecast the test set
forecast_steps = len(test)
forecast = sarima_result.get_forecast(steps=forecast_steps)
forecast_index = test.index
forecast_series = pd.Series(forecast.predicted_mean, index=forecast_index)
forecast_ci = forecast.conf_int()

# Plot the train, test, and forecast
plt.figure(figsize=(14, 7))
plt.plot(train['Sales_Count'], label='Train')
plt.plot(test['Sales_Count'], label='Test', color='orange')
plt.plot(forecast_series, color='red', label='Forecast')
plt.fill_between(forecast_index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], color='pink', alpha=0.3)
plt.title('Train/Test Split with SARIMA Forecast')
plt.xlabel('Date')
plt.ylabel('Number of Sales')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Print forecast accuracy metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(test['Sales_Count'], forecast_series)
mae = mean_absolute_error(test['Sales_Count'], forecast_series)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')


In [ ]:
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings("ignore")

weekly_sales= backup_weekly_sales.copy()
weekly_sales = weekly_sales.drop('Sales_MA', axis=1)
weekly_sales.set_index('Week_Start', inplace=True)

# Define the p, d, q parameters to take any value between 0 and 2
p = d = q = range(0, 3)
# Generate all different combinations of p, d and q triplets
pdq = list(itertools.product(p, d, q))
# Generate all different combinations of seasonal p, d and q triplets with seasonal period 52
seasonal_pdq = [(x[0], x[1], x[2], 52) for x in pdq]

best_aic = float("inf")
best_pdq = None
best_seasonal_pdq = None

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = SARIMAX(weekly_sales['Sales_Count'], order=param, seasonal_order=param_seasonal, enforce_stationarity=False, enforce_invertibility=False)
            results = mod.fit()
            if results.aic < best_aic:
                best_aic = results.aic
                best_pdq = param
                best_seasonal_pdq = param_seasonal
        except:
            continue

print(f'Best ARIMA Parameters: {best_pdq} with seasonal {best_seasonal_pdq}')


In [ ]:
def is_in_range(laua):
    try:
        # Extract the numeric part and convert to integer
        laua_num = int(laua[1:])
        # Check if it is within the range
        return 9000001 <= laua_num <= 9000033
    except ValueError:
        return False

# Apply the function to the 'laua' column
filtered_nspl_df = nspl_df[nspl_df['laua'].apply(is_in_range)]

# Display the filtered DataFrame
print("Records where 'laua' values are between E09000001 and E09000033:")
print(filtered_nspl_df)

In [ ]:
df.info()

In [ ]:
df_bacckup = df.copy()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df[df['Price']>8.0e+08]

In [ ]:
filtered = df[
    (df['Postcode'] == 'TN23 7HE')

]

In [ ]:
filtered

In [ ]:
 Category_B = df[df['PPD_Category_Type']=='B']
 Category_B_sorted = Category_B.sort_values(by='Date_of_Transfer')

In [ ]:
Category_B['PPD_Category_Type'].value_counts()

In [ ]:
cutoff_date = pd.Timestamp('2013-10-01')
Category_B_sorted[Category_B_sorted['Date_of_Transfer'] < cutoff_date]

In [ ]:
categorical_columns = [
    'Property_Type',
    'Old/New',
    'Duration',
    'PPD_Category_Type',
    'Record_Status_monthly_file_only'
]

Address_columns = [
    'SAON',
    'PAON',
    'Street',
    'Locality',
    'Town/City',
    'District',
    'County',
    'Postcode'
]

Transaction_columns = [
    'Transaction_unique_identifier',
    'Price',
    'Date_of_Transfer'
]

In [ ]:
duplicate = df[df.duplicated(subset=['SAON',
    'Property_Type',
    'Old/New',
    'Duration',
    'PPD_Category_Type',
    'PAON',
    'Street',
    'Locality',
    'Town/City',
    'District',
    'County',
    'Postcode',
    'Date_of_Transfer'], keep=False)]

In [ ]:
duplicate

In [ ]:
# Define the specific values to check
record_to_check = {
    'SAON': 'APARTMENT 5',
    'PAON': '100',
    'Street': 'SILLAVAN WAY',
    'Locality': pd.NA,
    'Town/City': 'BARNET',
    'District': 'BARNET',
    'County': 'GREATER MANCHESTER',
    'Postcode': 'CF63 1BB'
}

In [ ]:
# Filter the DataFrame for the specific record
filtered_df = df[
    (df['Town/City'] == record_to_check['Town/City'])
]

# Display the filtered DataFrame
print("Records matching the specified values:")
filtered_df

In [ ]:
duplicate_rows = address_df.duplicated()

In [ ]:
duplicate_entries = df[df.duplicated(subset=['Date_of_Transfer', 'Postcode','SAON', 'PAON','Street','Locality','Town/City','District','County', 'Property_Type', 'Old/New', 'Price'], keep=False)]
print("Potential duplicate records:")
duplicate_entries

In [ ]:
inconsistencies = df[(df['Old/New'] == 'N') & (df['Date_of_Transfer'] < '2000-01-01')]
print("Potential inconsistent records:")
inconsistencies

In [ ]:
df[dup]

In [ ]:
duplicated_df = df[duplicate_rows]

In [ ]:
duplicated_df['Duration'].value_counts()

In [ ]:
# Function to standardize string comparison
def standardize_string(s):
    if pd.isna(s):
        return s
    return str(s).strip().upper()

# Apply the function to the DataFrame and the record to check
for key in record_to_check.keys():
    df[key] = df[key].apply(standardize_string)
    record_to_check[key] = standardize_string(record_to_check[key])

In [ ]:
lease_houses = df[df['Duration'] == 'L']

# Find duplicate addresses and calculate the date difference
lease_houses_sorted = lease_houses.sort_values(by=['PAON', 'SAON', 'Postcode', 'Date_of_Transfer'])
lease_houses_sorted['Date_Diff_Days'] = lease_houses_sorted.groupby(['PAON', 'SAON', 'Postcode'])['Date_of_Transfer'].diff().dt.days

# Display the DataFrame with the new 'Date_Diff_Days' column
print("Lease houses with date difference in days:")
lease_houses_sorted

In [ ]:
lease_houses_sorted[lease_houses_sorted['Date_Diff_Days']<2555]

In [ ]:
grouped_duplicates = duplicates.groupby(Address_columns).size().reset_index(name='Counts')

In [ ]:
grouped_duplicates1 = grouped_duplicates[grouped_duplicates['Counts'] > 1]
grouped_duplicates1

In [ ]:
plt.figure(figsize=(20, 10))

# Loop through the categorical columns and plot the value counts
for i, column in enumerate(categorical_columns, 1):
    plt.subplot(2, 3, i)
    sns.countplot(y=df[column], order=df[column].value_counts().index)
    plt.title(f'Value Counts of {column}')
    plt.xlabel('Counts')
    plt.ylabel(column)

plt.tight_layout()
plt.show()

In [ ]:
df['Duration'].value_counts()

In [ ]:
df['Old/New'].value_counts()

In [ ]:
print(len(df))
df['unknown'].value_counts()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df1 = df.copy()

In [ ]:
df1['Price'].describe()

In [ ]:
df1[df1['Price']<100]

In [ ]:
duplicated_transactions = df1[df1.duplicated(subset=['Transaction_unique_identifier'], keep=False)]

# Check if there are any duplicates and display them
if not duplicated_transactions.empty:
    print("Duplicates found in Transaction unique identifier:")
    print(duplicated_transactions)
else:
    print("No duplicates found in Transaction unique identifier.")

In [ ]:
duplicate_entries = df1[df1.duplicated(subset=["PAON","SAON","Street","Locality","Town/City","District","County","Postcode"], keep=False)]
print("Potential duplicate records:")
print(duplicate_entries)

In [ ]:
gdf1 = gpd.read_file('/content/drive/MyDrive/Chimnie/OSGB_Grids-master/GeoJSON/OSGB_Grid_10km.geojson')

In [ ]:
gdf1.info()

In [ ]:
print(gdf1.crs)

In [ ]:
def determine_region(row):
    if row['ENGLAND'] == 't':  # Assuming '1' indicates presence
        return 'England'
    elif row['SCOTLAND'] == 't':  # Assuming '1' indicates presence
        return 'Scotland'
    elif row['WALES'] == 't':  # Assuming '1' indicates presence
        return 'Wales'
    else:
        return 'Other'

gdf1['REGION'] = gdf1.apply(determine_region, axis=1)

# Plotting the GeoDataFrame with different colors for each region
fig, ax = plt.subplots(figsize=(10, 10))
gdf1.plot(column='REGION', ax=ax, legend=True, cmap='Set3', edgecolor='k')

# Setting title and labels
ax.set_title('10km Map of England, Scotland, and Wales')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Show plot
plt.show()